# Problem 1. [60 points] Making Sandcastle

Grad student Alice is at Cancun to spend her end-of-the-year vacation. Relaxing at the beautiful white sand beach, she contemplates making a sandcastle while minimizing her effort to construct it. 

Alice approximates the sandy beach as part of a 2D plane. She decides to move the sand mass from $m$ known source locations $x_{i}^{\text{source}}\in\mathbb{R}^{2}$, $i=1,...,m$, to $n$ known destination locations $x_{j}^{\text{sandcastle}}\in\mathbb{R}^{2}$ , $j=1,...,n$. 

Sand particles in the beach have nonuniform mass. In particular, the $i$th source location has a known mass $\alpha_i>0$, $i=1,...,m$. Alice's sandcastle design requires the $j$th destination location to have a known mass $\beta_j>0$, $j=1,...,n$. Conservation of mass requires $\sum_{i}\alpha_i = \sum_{j}\beta_{j}$. Without loss of generality, Alice normalizes the mass, i.e., sets $\sum_{i}\alpha_i = \sum_{j}\beta_{j}=1$. In other words, known $\alpha_i$ denotes the fraction of total mass at the $i$th source location. Similar interpretation holds for $\beta_{j}$.

Alice models the cost $C_{ij}$ of moving **unit amout of sand** from the source location $x_{i}^{\text{source}}$ to the destination location $x_{j}^{\text{sandcastle}}$ as squared Euclidean distance, i.e., $C_{ij} = \|x_{i}^{\text{source}} - x_{j}^{\text{sandcastle}}\|_{2}^{2}$. This defines a matrix $C\equiv [C_{ij}]\in\mathbb{R}^{m\times n}$. 

## (a) [5 + 10 + (5 + 5) + 10 = 35 points] Formulation

(i) **Explain why** Alice's model for $C_{ij}$ is reasonable. 

(ii) If Alice decides to move $M_{ij}$ amount of mass from $x_{i}^{\text{source}}$ to $x_{j}^{\text{sandcastle}}$, then she incurs a cost $C_{ij}M_{ij}$ for that particular route. Taking the matrix $M\equiv [M_{ij}]\in\mathbb{R}^{m\times n}$ as the decision variable, **clearly write down the optimization problem** Alice needs to solve for making her sandcastle. The input parameters for the problem should be the matrix $C\in\mathbb{R}^{m\times n}$, the vector $\alpha\in\mathbb{R}^{m}_{++}$, and the vector $\beta\in\mathbb{R}^{n}_{++}$. Assume that the problem data already guarantees $\langle\boldsymbol{1}_{m},\alpha\rangle = \langle\boldsymbol{1}_{n},\beta\rangle = 1$ where $\boldsymbol{1}_{m},\boldsymbol{1}_{n}$ denote the vector of all ones of size $m\times 1$ and $n\times 1$, respectively.

(iii) **Mathematically explain why** this is a convex optimization problem. **Mathematically argue what type** of convex optimization problem is this.

Side remark: Unlike the exercises in HW5 Problem 2, this problem has no analytical solution in terms of the problem data.

(iv) **Carefully argue the size of the optimization problem**, i.e., how many variables are to be solved for and how many constraints are there.

## (b) [25 points] Numerical solution

Fix $m=150$, $n=225$. Write a code in MATLAB/Python/Julia to load the input from CANVAS Files section: HW problems and solutions: alpha.txt, beta.txt, x_source.txt, x_sandcastle.txt, and use cvx/cvxpy/Convex.jl in the same code to solve the optimization problem in part (a). Report the **numerically computed optimal value (minimized cost) and submit your code**. **Also report the computational time needed to solve the problem by cvx/cvxpy/Convex.jl** (only the computational time for solving, not for setting up the problem data).

Side remark: It is recommended (but not required) that in your code, you also check if your optimal solution obtained from using cvx/cvxpy/Convex.jl matches with linprog in MATLAB, or with scipy.optimize.linprog in Python.

i.
1. Defined as $C_{ij} = \|x_{i}^{\text{source}} - x_{j}^{\text{sandcastle}}\|_{2}^{2}$, every element $C_{ij}$ of the rectangular matrix $C$ is **non-negative** and **convex**.

ii.
\begin{align}
p^{*} = &{\min} \quad \langle C, M \rangle \\
&\text{subject to} \quad 0 \leq M_{ij} \\
&\text{subject to} \quad \langle E_i, M \rangle = \alpha_i, \quad\forall\,i=1,...,m, \\
&\text{subject to} \quad \langle D_j, M \rangle = \beta_j, \quad\forall\,j=1,...,n, \\
&\text{where $E_i$ is [0] matrix, 1 along row $i$} \\
&\text{where $D_j$ is [0] matrix, 1 along column $j$} \\
\end{align}

iii.
1. The objective function is a **linear function in $x$**, (another form of it is to convert $C$ to a vector $\underline{c}$ by concatenating columns of $C$ into a vector $\underline{c}$). Then same can be done for $M$ to form the vector $\underline{m}$, then the frobenius inner product becomes equivalent to $\underline{c}^{T} \underline{m}$
2. The constraint sets is a **polyhedron**, of $m + n$ equalities and $m \times n$ inequalities.
  * The equalities come from the vectorizing $E_i$ into a vector $\underline{e_i}$ and $\underline{e_i}^{T} \underline{m} = \alpha_i$ and likewise for $D_j$.
  * The halfspace / inequalities come from $a_{ij}$ being the null vector except -1 at the $m \times i + j$ location, such that $a_{ij}^{T} \underline{m} = -m_{ij} \leq 0 \Leftrightarrow m_{ij} \geq 0$
3. Then from lec 11 pg 4, we know that this is a convex optimization problem, specifically a **linear program** because of the reasons above

iv.
1. Constraints are covered above, there are $m + n$ equalities, $m \times n$ inequalities, and there are $m \times n$ decision variables in the matrix $M$ or vector $\underline{m}$

In [41]:
import cvxpy as cp
import numpy as np
from scipy.spatial.distance import cdist
from scipy import sparse
import time

alpha = None
with open('./alpha.txt') as f: 
    alpha = np.array([float(x) for x in f.read().splitlines()])
beta = None
with open('./beta.txt') as f: 
    beta = np.array([float(x) for x in f.read().splitlines()])
x_source = None
with open('./x_source.txt') as f: 
    x_source = f.read().splitlines()
    x_source = [[float(y) for y in x.split("\t")] for x in x_source]
x_source = np.array(x_source)
    
x_sandcastle = None
with open('./x_sandcastle.txt') as f: 
    x_sandcastle = f.read().splitlines()
    x_sandcastle = [[float(y) for y in x.split("\t")] for x in x_sandcastle]
x_sandcastle = np.array(x_sandcastle)
    
m = len(alpha)
n = len(beta)

print(len(alpha), len(beta))
print(np.sum(alpha))
print(np.sum(beta))

C = cdist(x_source, x_sandcastle, 'sqeuclidean')
cvector = C.reshape(m*n)

# row equalities
e_stacked = np.kron(
    np.ones((1,n)), # pick that column's element for every column
    sparse.eye(m).toarray() # an eye picks that column's element for every row
)

# column equalities
d_stacked = np.kron(
    sparse.eye(n).toarray(), # 1 for every col, so eye(n)
    np.ones((1,m)) # 1 for every row in column, so m 1s in a row
)

print("e_stacked.shape", e_stacked.shape)
print("d_stacked.shape", d_stacked.shape)

col_major_m = cp.Variable(m * n, nonneg=True)
prob = cp.Problem(
    cp.Minimize(cvector @ col_major_m),
    [
        e_stacked @ col_major_m == alpha,
        d_stacked @ col_major_m == beta,
    ])

start = time.time()
prob.solve(verbose=True, solver='ECOS_BB')
end = time.time()
print("solution", prob.value)

start = time.time()
prob.solve(verbose=True, solver='SCS')
end = time.time()
print("solve time", end - start)

150 225
0.9999999995
1.0000000003
e_stacked.shape (150, 33750)
d_stacked.shape (225, 33750)
                                     CVXPY                                     
                                    v1.1.18                                    
(CVXPY) Nov 12 12:00:46 PM: Your problem has 33750 variables, 2 constraints, and 0 parameters.
(CVXPY) Nov 12 12:00:46 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Nov 12 12:00:46 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Nov 12 12:00:46 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Nov 12 12:00:46 PM: Compiling problem (target solver=ECOS_BB)

------------------------------------------------------------------
	       SCS v3.1.0 - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012
------------------------------------------------------------------
problem:  variables n: 33750, constraints m: 34125
cones: 	  z: primal zero / dual free vars: 375
	  l: linear vars: 33750
settings: eps_abs: 1.0e-05, eps_rel: 1.0e-05, eps_infeas: 1.0e-07
	  alpha: 1.50, scale: 1.00e-01, adaptive_scale: 1
	  max_iters: 100000, normalize: 1, warm_start: 0
	  acceleration_lookback: 10, acceleration_interval: 10
lin-sys:  sparse-direct
	  nnz(A): 101250, nnz(P): 0
------------------------------------------------------------------
 iter | pri res | dua res |   gap   |   obj   |  scale  | time (s)
------------------------------------------------------------------
     0| 3.27e+01  1.69e+01  1.18e+06 -5.90e+05  1.00e-01  6.19e-03 
   250| 1.18e-05  1.10e-01  1.19e-02  4.47e-01  3.53e+03  2.70e-01 
   500| 1.95e-06  2.90e-02  1.76e-0

# Problem 2. [15 + 15 + 10 = 40 points] Lagrange Dual Problem

Consider the primal convex optimization problem
\begin{align}
p^{*} = &\underset{x\in\mathbb{R}^{n}}{\min}\quad\frac{1}{2}x^{\top}P_{0}x + \langle q_0, x\rangle + r_0\\
&\text{subject to} \quad \frac{1}{2}x^{\top}P_{i}x + \langle q_i, x\rangle + r_i \leq 0, \quad\forall\,i=1,...,m,
\end{align}
where $P_0\in\mathbb{S}^{n}_{++}$, $P_{i}\in\mathbb{S}^{n}_{+}$ for all $i=1,...,m$, $q_i\in\mathbb{R}^{n}$ for all $i=0,1,...,m$, and $r_i\in\mathbb{R}$ for all $i=0,1,...,m$.

(a) Denote the Lagrange multiplier associated with the primal inequality constraints as $\lambda\in\mathbb{R}^{m}_{\geq 0}$. Let
\begin{align}
P(\lambda) := P_{0} + \sum_{i=1}^{m}\lambda_i P_i \succ 0,\quad q(\lambda) &:= q_0 + \sum_{i=1}^{m}\lambda_i q_i,\quad r(\lambda) := r_0 + \sum_{i=1}^{m}\lambda_i r_i.
\end{align}
**Prove that** the Lagrange dual problem associated with the primal problem is 
$$d^{*} = \underset{\lambda\in\mathbb{R}^{m}_{\geq 0}}{\min}\:\frac{1}{2}\left(q(\lambda)\right)^{\top}\left(P(\lambda)\right)^{-1}q(\lambda) - r(\lambda).$$

(b) Rewrite the Lagrange dual problem derived in part (a) in one of the standard forms: LP, QP, QCQP, SOCP, SDP. **Show all your calculations**.

(c) Specialize Slater's condition for this primal problem to **state a sufficient condition for strong duality** ($p^{*}=d^{*}$).

a
1. Derive the **Lagrangian**
\begin{align}
\mathcal{L}(\underline{x}, \underline{\lambda}) & = \frac{1}{2}x^{\top}P_{0}x + \langle q_0, x\rangle + r_0 + \langle \lambda, \frac{1}{2}x^{\top}P_{i}x + \langle q_i, x\rangle + r_i \rangle \\
& = \frac{1}{2}x^{\top}P_{0}x + \langle q_0, x\rangle + r_0 + \sum_{i=1}^{m}\lambda_i \frac{1}{2}x^{\top}P_{i}x + \sum_{i=1}^{m}\lambda_i \langle q_i, x\rangle + \sum_{i=1}^{m}\lambda_i r_i \\
& = \frac{1}{2}x^{\top}P_{0} x + \frac{1}{2} \sum_{i=1}^{m} x^{\top}\lambda_i P_{i} x + \langle q_0, x\rangle + \sum_{i=1}^{m} \langle \lambda_i q_i, x\rangle + r_0 + \sum_{i=1}^{m}\lambda_i r_i \\
& = \frac{1}{2}x^{\top}P_{0} x + \frac{1}{2} \sum_{i=1}^{m} x^{\top}\lambda_i P_{i} x + \langle q_0, x\rangle + \sum_{i=1}^{m} \langle \lambda_i q_i, x\rangle + r(\lambda) \\
& = \frac{1}{2}x^{\top} P(\lambda) x + \langle q(\lambda), x\rangle + r(\lambda) \\
\end{align}
2. Then for the **dual function**, we take the derivative and equate to 0 to derive $\underline{x}_{opt}$
\begin{align}
g(\lambda) & = \underset{\underline{x} \in \mathbb{R}^{n}}\inf{L(\underline{x}, \underline{\lambda})}\\
\frac{\partial}{\partial{x}} L(\underline{x}, \underline{\lambda}) & = P(\lambda)\underline{x} + q(\lambda) \\
\Rightarrow \underline{x}_{opt} & = -(P(\lambda))^{-1} q(\lambda) \\
\Rightarrow g(\lambda) & = \frac{1}{2} (-P^{-1} q)^{\top} P (- P^{-1} q) + q^{\top} (-P^{-1} q) + r \\
& = \frac{1}{2} (P^{-1} q)^{\top} q - q^{\top} P^{-1} q + r \\
& = \frac{1}{2} q^{\top} (P^{-1})^{\top} q - q^{\top} P^{-1} q + r \\
& = \frac{1}{2} q^{\top} P^{-1} q - q^{\top} P^{-1} q + r \quad \text{P symmetric}\\
& = -\frac{1}{2} q^{\top} P^{-1} q + r \\
\end{align}
3. The dual is thus (Lec 14 pg 5 proves the domain of $\underline{\lambda} \in \mathbb{R}^{m} \geq 0$)
\begin{align}
\underset{\underline{\lambda} \in \mathbb{R}^{m} \geq 0}\sup{g(\lambda)} & = \underset{\underline{\lambda} \in \mathbb{R}^{m} \geq 0}\min{-g(\lambda)}\\
& = \underset{\underline{\lambda} \in \mathbb{R}^{m} \geq 0}\min{\frac{1}{2} q(\lambda)^{\top} (P(\lambda))^{-1} q(\lambda) - r(\lambda)}$
\end{align}

b.
1. We can use the 'turn objective function to constraint' trick in Lec 13 pg 17 to turn the non-linear objective function into a linear constraint.
\begin{align}
d^{*} & = \underset{\underline{\lambda}, t}\min{t} \\
&\text{subject to}\\
&\quad \frac{1}{2}q(\lambda)^{\top}P(\lambda)^{-1}q(\lambda) - r(\lambda) \leq t \\
& \Leftrightarrow \frac{1}{2}q(\lambda)^{\top}P(\lambda)^{-1}q(\lambda) - r(\lambda) - t \leq 0 \\
& \Leftrightarrow \mathcal{S} = t + r(\lambda) - \frac{1}{2}q(\lambda)^{\top}P(\lambda)^{-1}q(\lambda) \geq 0 \\
\end{align}
2. We see that $\mathcal{S}$ is the **Schur complement** of
\begin{align}
\mathcal{X} & = \begin{bmatrix} 2P(\lambda) \succ 0 & q(\lambda) \\ q(\lambda)^{\top} & t + r(\lambda) \end{bmatrix} 
\end{align}
3. Then from lec 9 pg 3, since the top left $A = 2P(\lambda) \succ 0$, $\mathcal{S} \succeq 0 \Leftrightarrow \mathcal{X} \succeq 0$
4. So then we see that the dual $d^{*}$ has been rewritten as (lec 11 pg 10) **SDP** standard form:
\begin{align}
d^{*} & = \underset{\underline{\lambda}, t}\min{t} \\
&\text{subject to}\\
& \mathcal{X} = \begin{bmatrix} 2P(\lambda) \succ 0 & q(\lambda) \\ q(\lambda)^{\top} & t + r(\lambda) \end{bmatrix} \succeq 0\\
\end{align}

c.
1. Since the $f_{i}(x)$ constraints are **nonlinear**, Slater's condition for this problem says the sufficient condition is if:
\begin{align}
\exists \underline{x}^{*} & \in \text{relint}(\text{dom}(p^{*})) \\
f_{i}(\underline{x}^{*}) & = \frac{1}{2} (\underline{x}^{*})^{\top}P_{i}\underline{x}^{*} + \langle q_i, \underline{x}^{*}\rangle + r_i \lt 0 \quad \forall i=1,...m \\
\Rightarrow \langle q_i, \underline{x}^{*}\rangle + r_i & \lt -\frac{1}{2} (\underline{x}^{*})^{\top}P_{i}\underline{x}^{*} \leq 0 \quad \forall i=1,...m \\
\end{align}